# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

F. Walter  ->  F. Walter  |  ['F. Walter']
F. Bruckmann  ->  F. Bruckmann  |  ['F. Bruckmann']
E. Bañados  ->  E. Bañados  |  ['E. Bañados']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']


Arxiv has 77 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2510.01314


extracting tarball to tmp_2510.01314...

 done.
  0: tmp_2510.01314/aassymbols.tex, 579 lines
  1: tmp_2510.01314/main.tex, 390 lines
Retrieving document from  https://arxiv.org/e-print/2510.01421


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found 2 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Assuming tmp_2510.01314/aassymbols.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)


...

extracting tarball to tmp_2510.01486...

 done.


Found 115 bibliographic references in tmp_2510.01486/demo_caosp310.bbl.
Issues with the citations
syntax error in line 67: '=' expected
Retrieving document from  https://arxiv.org/e-print/2510.01872


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem[{{Benati Gon{ç}alves} {et~al.}(2025){Benati Gon{ç}alves}, {Panda}, {Storchi Bergmann}, {Cackett},  {Eracleous}}]{Benati-Goncalves2025ApJ}{Benati Gon{ç}alves}, H., {Panda}, S., {Storchi Bergmann}, T., {Cackett}, E.~M.,  {Eracleous}, M., {Exploring Quasar Variability with ZTF at 0 < z < 3: A Universal Relation with the Eddington Ratio}. 2025, {\it \apj}, {\bf 988}, 27, \href{https://doi.org/10.3847/1538-4357/addec0}{\path{DOI: 10.3847/1538-4357/addec0}}%ADS_ID Bentz_2013
 regex = 
        \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3}).*?href(.*?{(?P<url>http[\S]*)})(?P<rest>.*)
        
  warnings.warn(str(e))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem[{{Besuner} {et~

extracting tarball to tmp_2510.01872... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.01486-b31b1b.svg)](https://arxiv.org/abs/2510.01486) | **Feeding frenzy in the mighty black holes: what we could learn from them?**  |
|| S. Panda, et al. -- incl., <mark>E. Bañados</mark> |
|*Appeared on*| *2025-10-03*|
|*Comments*| *30 pages, 8 figures, review based on the invited talk at the 15th Serbian Conference on Spectral Line Shapes in Astrophysics*|
|**Abstract**|            Eddington ratio is a paramount parameter governing the accretion history and life cycles of Active Galactic Nuclei (AGNs). This short review presents a multi-faceted view of the importance of the Eddington ratio spanning varied AGN studies. We find that the Eddington ratio is crucial for standardizing the Radius-Luminosity (R-L) relation - a necessary step for employing quasars (QSOs) as standardizable cosmological probes to help clarify the standing of the Hubble tension. In this data-driven era, we consolidated disparate aspects by developing novel relations borne out of large datasets, such as the robust, nearly universal anti-correlation between fractional variability and Eddington ratio derived from Zwicky Transient Facility (ZTF) data, which is vital for interpreting forthcoming high-cadence surveys like Rubin Observatory's LSST. Addressing the conundrum where JWST results suggest an overabundance of massive high-redshift black holes, we demonstrate that local AGNs offer clarification: Changing-Look AGNs (CLAGNs), driven by rapid Eddington ratio shifts, cluster in the low-accretion regime, a rate independently confirmed by our integral field spectroscopy and photoionization modeling of a well-known Seyfert 2 galaxy, rich in high-ionization, forbidden, coronal lines. Conversely, for the high-redshift, high-luminosity population where traditional reverberation mapping (RM) is highly impractical, photometric reverberation mapping (PRM) offers a rapid alternative to constrain accretion disk sizes, enabling efficient estimates of black hole masses and Eddington ratios. Finally, we developed tailored semi-empirical spectral energy distributions (SEDs) for extremely high-accretion quasars, successfully validating their characteristic extreme physical conditions.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.01314-b31b1b.svg)](https://arxiv.org/abs/2510.01314) | **JWST Observations of Starbursts: PAHs Closely Trace the Cool Phase of M82's Galactic Wind**  |
|| S. Lopez, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2025-10-03*|
|*Comments*| *14 pages, 7 figures, submitted to ApJ Letters (01 October 2025)*|
|**Abstract**|            Stellar feedback drives multiphase gas outflows from starburst galaxies, but the interpretation of dust emission in these winds remains uncertain. To investigate this, we analyze new JWST mid-infrared images tracing polycyclic aromatic hydrocarbon (PAH) emission at 7.7 and 11.3~$\mu$m from the outflow of the prototypical starburst M82 out to $3.2$ kpc. We find that PAH emission shows significant correlations with CO, H$\alpha$, and X-ray emission within the outflow, though the strengths and behaviors of these correlations vary with gas phase and distance from the starburst. PAH emission correlates strongly with cold molecular gas, with PAH--CO scaling relations in the wind nearly identical to those in galaxy disks despite the very different conditions. The H$\alpha$--PAH correlation indicates that H$\alpha$ traces the surfaces of PAH-bearing clouds, consistent with arising from ionized layers produced by shocks. Meanwhile the PAH--X-ray correlation disappears once distance effects are controlled for past 2~kpc, suggesting that PAHs are decoupled from the hot gas and the global correlation merely reflects the large-scale structure of the outflow. The PAH-to-neutral gas ratio remains nearly flat to 2~kpc, with variations following changes in the radiation field. This implies that the product of PAH abundance and dust-to-gas ratio does not change significantly over the inner portion of the outflow. Together, these results demonstrate that PAHs robustly trace the cold phase of M82's wind, surviving well beyond the starburst and providing a powerful, high-resolution proxy for mapping the life cycle of entrained cold material in galactic outflows.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.01421-b31b1b.svg)](https://arxiv.org/abs/2510.01421) | **Ly-alpha emission reveals two satellite halos around massive groups at z ~ 3: the puzzling case of a quiescent central galaxy**  |
|| S. Guo, et al. -- incl., <mark>F. Bruckmann</mark> |
|*Appeared on*| *2025-10-03*|
|*Comments*| *Resubmitted to A&A after addressing referee comments*|
|**Abstract**|            We present the discovery and characterisation of two Ly$\alpha$ nebulae (LANs), RO-1001-Sat and RO-0959-Sat, as satellite structures of two giant LANs at $z=2.920$ and 3.092. They are found neighbouring two out of four known giant LANs at $z\sim3$ in our MUSE follow-up observations, reinforcing the idea that Ly$\alpha$ emission can be used to trace massive dark matter halos at high-$z$. This high occurrence of massive satellite halos agrees with simulations. With sizes of $\simeq80\times160$ and $80\times100~\mathrm{pkpc}^2$, the two nebulae are both $\sim$300pkpc from the main LANs. The Ly$\alpha$ emission is only shifted by $\simeq100-300$ km s$^{-1}$ between each of the two pairs, suggesting connections via large-scale structure. RO-1001-Sat and RO-0959-Sat are estimated to have log$(M_\mathrm{h}/M_\odot)\simeq13.2\pm0.3$ and $12.8\pm0.3$, putting them potentially close to the regime of cold-mode accretion. The central brightest galaxies in the two halos are morphologically distinct despite having similar stellar mass $\sim10^{11}M_\odot$, one being an elliptical quiescent galaxy in RO-1001-Sat and the other being a dusty star-forming spiral in RO-0959-Sat. Intriguingly, the quiescent galaxy aligns well with the peak of the LAN as well as the potential well of the host halo, making it the first clear-cut case where the cold gas ought to be accreting onto the galaxy but with no observable star formation, either due to morphological quenching or, more likely, radio-mode feedback from an active galactic nucleus. Finally, we show a tentative detection of a Ly$\alpha$ filament connecting RO-1001 and RO-1001-Sat. This work shows how panoramic MUSE (and in the future, BlueMUSE) observations of massive halo seeds can be used to efficiently search for additional halos, unveiling their large-scale structure and enabling the study of Ly$\alpha$-selected galaxy groups.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.01872-b31b1b.svg)](https://arxiv.org/abs/2510.01872) | **Constraining the Neutron-Star Equation of State via Short Gamma-Ray Burst X-ray Afterglows**  |
|| R. Moradi, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-10-03*|
|*Comments*| *Accepted for publication in The Astrophysical Journal (ApJ)*|
|**Abstract**|            Recent observations from NICER in X-rays and LIGO/Virgo in gravitational waves have provided critical constraints on the mass, radius, and tidal deformability of neutron stars, imposing stringent limits on the equation of state (EOS) and the behavior of ultra-dense matter. However, several key parameters influencing the EOS, such as the maximum mass of neutron stars, spin-down rates, and the potential role of exotic matter in their cores, remain subject of ongoing debate. Here we present a new approach to constraining the EOS by analyzing the X-ray afterglows of some short gamma-ray bursts, focusing on "the internal plateau" phase and its abrupt decay, which reflect the spin-down and possible collapse of a supra-massive neutron star into a black hole. By linking critical neutron star masses with black hole formation criteria and the observational data from Swift's BAT and XRT instruments with compact object models, we explore three representative EOSs that range from "soft" to "stiff". Our result supports a maximum mass for neutron stars of approximately 2.39 solar masses at the threshold of black hole formation. This conclusion holds under assumptions of magnetar-powered X-ray plateaus, constant radiative efficiency, isotropic emission, and full Kerr black hole energy extraction; deviations could influence the inferred results. Our results demonstrate the critical role of neutron star/black hole physics in probing dense nuclear matter and provide a novel framework for exploring extreme astrophysical environments.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2510.01486/./original_correlation.png', 'tmp_2510.01486/./Eddr_dist.png', 'tmp_2510.01486/./CLAGN_trend.png', 'tmp_2510.01486/./eso_g138.png']
copying  tmp_2510.01486/./original_correlation.png to _build/html/
copying  tmp_2510.01486/./Eddr_dist.png to _build/html/
copying  tmp_2510.01486/./CLAGN_trend.png to _build/html/
copying  tmp_2510.01486/./eso_g138.png to _build/html/
exported in  _build/html/2510.01486.md
    + _build/html/tmp_2510.01486/./original_correlation.png
    + _build/html/tmp_2510.01486/./Eddr_dist.png
    + _build/html/tmp_2510.01486/./CLAGN_trend.png
    + _build/html/tmp_2510.01486/./eso_g138.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\BibTeX}{{\rm B\kern-.05em{\sc i\kern-.025em b}$
$             T\kern-.1667em\lower.7ex\hbox{E}\kern-.125emX}}$</div>



<div id="title">

# Feeding frenzy in the mighty black holes:\ what we could learn from them

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.01486-b31b1b.svg)](https://arxiv.org/abs/2510.01486)<mark>Appeared on: 2025-10-03</mark> -  _30 pages, 8 figures, review based on the invited talk at the 15th Serbian Conference on Spectral Line Shapes in Astrophysics_

</div>
<div id="authors">

S. Panda, et al. -- incl., <mark>E. Bañados</mark>

</div>
<div id="abstract">

**Abstract:** * Data-driven era, need for us to consolidate/learn from/stitch the different aspects together; a bit of out-of-the-box thinking and novel relations borne out of the analyses of large datasets and innovative techniques\end{itemize}\end{comment}$ $\keywords{Supermassive black holes (1663) -- Active galactic nuclei (16) -- Quasars (1319) -- Spectroscopy (1558) -- Photometry (1234) -- Scaling relations (2031) -- Spectral energy distribution (2129) -- Photoionization (2060)}$

</div>

<div id="div_fig1">

<img src="tmp_2510.01486/./original_correlation.png" alt="Fig1" width="100%"/>

**Figure 1. -** Distribution of the fractional variability (F$_{\rm var}$) in the g-band ZTF lightcurves for the AQMES medium field monitored within the SDSS-V, versus the Eddington ratio. The latter is taken from the SDSS DR16 QSO catalogue \citep{Wu_Shen_2022ApJS..263...42W}. The color axis depicts the distribution of redshift. The best-fit correlation after cleaning sources with insufficient F$_{\rm var}$ in g-band information: log $\lambda_{\rm Edd}$ = -0.61 log F$_{\rm var}$ - 1.38 ($\rho$ = -0.28; p-value = 2.8E-18). (*fig1*)

</div>
<div id="div_fig2">

<img src="tmp_2510.01486/./Eddr_dist.png" alt="Fig2.1" width="50%"/><img src="tmp_2510.01486/./CLAGN_trend.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** (_Left:_) Distribution of Eddington ratios in the sample from \citet{PandaSniegowska2024ApJS}. We show the distributions for the earliest (in brown) and the latest (in purple) epochs for the sources in our sample. The median values for the two distributions (red = -1.99, blue = -1.685) are marked with vertical dashed lines. The magenta box marks the range of the Eddington ratio for NGC 5548, i.e., log $\lambda_{\rm Edd}$ = [-2.2, -1]. (_Right:_) The distribution of the emission line EW versus the AGN continuum luminosity. Here, we demonstrate the trend for two sources:  SDSS J141324.27+530527.0 \citep{Wang_2018ApJ...858...49W} with 72 spectral epochs over $\sim$15 years (5527 days), and NGC 5548 \citep{Bon_2018FrASS...5....3B, Panda2022AN, Panda2023BASBr} with more than 750 spectral epochs over $\sim$25 years (9624 days). For the former source (SDSS J141324.27+530527.0), we have taken the spectral data from the homogeneous fitting in \citet{PandaSniegowska2024ApJS}, which includes the Mg{\sc ii}, H$\beta$, and H$\alpha$ emission lines and the corresponding AGN continua nearest to these lines (at 3000Å, 5100Å, and 6000Å), as shown in the panel. For NGC 5548, we show the dataset from Panda et al. (in prep.), which is an updated version of the dataset provided in \citet{Bon_2018FrASS...5....3B}, where the authors analyzed the H$\beta$ region. The SDSS source shows a clear rise from a deep minimum to a high state in both Balmer lines. However, the Mg{\sc ii} shows a rather flat behavior - reminiscent of the Baldwin effect, suggesting the difference in ionization and response to the changing continuum levels. NGC 5548 data has a wealth of data, but the change in the source is rather gradual, and hence, a clear spike in the trend is not that prominent. (*fig:clagns*)

</div>
<div id="div_fig3">

<img src="tmp_2510.01486/./eso_g138.png" alt="Fig3" width="100%"/>

**Figure 3. -** (_Left:_)[O {\sc iii}]$\lambda$5007 emission image of ESO 138-G001 from the HST/WFPC2 \citep{HST2000ApJS..128..139F} with a small inset showing the details of the central part, where the nucleus is marked with a plus sign; (_Right:_) [O {\sc iii}]$\lambda$5007 emission image from SIFS after data treatment involving spatial re-sampling with quadratic interpolation followed by the Richardson-Lucy PSF deconvolution \citep[see][for more details]{SIFS2024MNRAS}. The green circles – with an aperture radius of 0.6 arcsec – denote the extraction regions of the spectra for the North-East (NE) knot (top left), the South-East (SE) blob (bottom left), and the nuclear region. The red circle denotes the PSF FWHM of 0.71 arcsec. (*fig:hst-sifs*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.01486"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

129  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

18  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

6  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
